In [14]:
# !pip install python-igraph
!pip show gensim

Name: gensim
Version: 0.13.4.1
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: UNKNOWN
Location: /Users/raminsoleymani/git/ECO/src/venv/lib/python2.7/site-packages
Requires: smart-open, numpy, scipy, six


In [1]:
import numpy as np
import math
from gensim import models
from random import random

from igraph import Graph 
from tqdm import tqdm


In [3]:
# Library Gensim (Word2Vec examples)
### LOAD GOOGLE WORD2VEC... takes a few minutes

# Load google's word2vec model
# languages
english = 'models/GoogleNews-vectors-negative300.bin.gz'
spanish = 'models/SBW-vectors-300-min5.bin.gz'
german = 'models/german.model'
# load
model = models.KeyedVectors.load_word2vec_format(english, binary=True)
model.init_sims(replace=True) # save memory
# print 'model memory size, gb:', model.estimate_memory()['total']/(math.pow(1024,3))

model memory size, gb:

AttributeError: 'KeyedVectors' object has no attribute 'estimate_memory'

In [14]:
# print len(model.vocab)
# w_ind = {}
# for index, w in enumerate(model.vocab):
#     w_ind[w] = index 

3000000


In [17]:
model.most_similar_cosmul('parent',topn=40)

[(u'Parent', 0.8029279112815857),
 (u'parents', 0.7569969296455383),
 (u'grandparent', 0.7551882863044739),
 (u'Corporation_Nasdaq_SRCE', 0.7521066665649414),
 (u'Aker_Kv\xe6rner_ASA', 0.7503856420516968),
 (u'sibling', 0.7440497875213623),
 (u'Bancorp_Nasdaq_HEOP', 0.73293137550354),
 (u'parental', 0.7309925556182861),
 (u'wholly_owned_Shenzhen_AIVtech', 0.7300399541854858),
 (u'parent_stepparent', 0.726965069770813),
 (u'child', 0.7261224985122681),
 (u'TABLE_####.T_-####/##', 0.7260121703147888),
 (u'Inc._NASDAQ_TCBI', 0.725272536277771),
 (u'kindergartener', 0.725018322467804),
 (u'visit_uns.com', 0.7231084108352661),
 (u'parenting', 0.7222963571548462),
 (u'TriCo_Bancshares_NASDAQ_TCBK', 0.7220449447631836),
 (u'brokersXpress_provides', 0.7209407091140747),
 (u'Otis_elevator_jet', 0.7202916145324707),
 (u'NASDAQ_BOFI_BofI', 0.7201634645462036),
 (u'ChinaNet_Online_Media', 0.7188237905502319),
 (u'adoptive_parent', 0.717492401599884),
 (u'Parents', 0.7167693376541138),
 (u'Nancy_Sa

In [19]:
model_sz = len(model.vocab)
kNN = 30

graph = Graph(directed=True)
graph.add_vertices(model_sz)

with tqdm(total = model_sz) as pbar:
    for index, w in enumerate(model.vocab):
        similars = model.most_similar_cosmul(w,topn=kNN)
        for dis in similars:
            word_index = w_ind[dis[0]]
            graph.add_edge(index, word_index, weight=dis[1])
        pbar.update(1)
    
summary(graph)

  0%|          | 7844/3000000 [3:00:57<1520:46:21,  1.83s/it]


KeyboardInterrupt: 

Some helper functions that print and return an word arithmetic for:
- equation (classic similarity)
- equation (cosmul similarity)
- word (cosmul similarity)

In [ ]:
pickle.dump([w_ind, graph], open('google_eng_30knn.p', 'wb'))

In [92]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

# simple utility functions for printing
def print_similarities(similarity_list):
    for similarity in similarity_list:
        print similarity[0].ljust(18),similarity[1]
    
def print_k_most_similar(similarity_list, k):
        print ', '.join(sim[0] for sim in similarity_list[:k])
        
def similarities_p(pos=[],neg=[],topn=10,doPrint = True):
    if doPrint: 
        print "pos:",pos,"neg:",neg
    similarities = model.most_similar(pos,neg,topn)
    if doPrint: print_similarities(similarities)
    return similarities
    
def most_similar_cosmul_p(pos=[],neg=[],topn=10, doPrint = True):
    if doPrint: print "( " + " + ".join(pos),") - " + "-".join(neg)
    similarities = model.most_similar_cosmul(pos,neg,topn)
#     if doPrint: print_similarities(similarities)    
    if doPrint: print_k_most_similar(similarities,5)    
    return similarities

def word_similar_cosmul_p(word,topn=10, doPrint = True):
    if doPrint: print "word:",word
    similarities = model.most_similar_cosmul([word],[],topn)
    if doPrint: print_similarities(similarities)    
    return similarities    

# logging.warning('hi')

In [93]:
# model['Frau']

In [114]:
# most_similar_cosmul_p(['Berlin','Haus'],['London'],10)
# most_similar_cosmul_p(['Berlin','Haus'],['Paris'],10)
# most_similar_cosmul_p(['Berlin','Currywurst'],['Muenchen'],10)
# most_similar_cosmul_p(['Berlin','Currywurst'],['Koeln'],10)
# most_similar_cosmul_p(['Berlin','Currywurst'],['Stuttgart'],10)
# most_similar_cosmul_p(['Berlin','Currywurst'],['Hamburg'],10)
most_similar_cosmul_p(['Kind','Huhn'],[],10)
print

( Kind ) - Huhn
Kindes, Geburt_Kindes, Kleinkind, Grundschulalter, Schuleintritt



In [7]:
# DEMO

# 1 word arithmetic
#model.most_similar(positive=["woman",'girl']) # gunshot?
print "WORD ARITHMETIC"
model.most_similar(positive=["Mann"]) 
print model.most_similar(positive=["Mann",'Junge']) 
# our helper fcts
similarities_p(['Deutschland'],['Spanien'],topn=5)
most_similar_cosmul_p(['Deutschland'],['Spanien'],topn=5)
# 2 similarity
print "\nSIMILARITY"
w1 = 'Frau'
w2 = 'Mann'
print model.similarity(w1,w2)

# 4 non-matching
print "\nNON-MATCHING WORD"
print model.doesnt_match("Frühstück Müsli Abendessen Mitagessen".split())

# 5 feature vector
vector = model['computer']
#print vector

# 6 similar by vector
vector[np.random.randint(0,300)] += random() * 0.5 - 0.05

print model.similar_by_vector(vector)

WORD ARITHMETIC
[(u'Bub', 0.7549524307250977), (u'Bursche', 0.7502634525299072), (u'junger_Mann', 0.7442663908004761), (u'Familienvater', 0.7426276206970215), (u'Frau', 0.7329385280609131), (u'Bursch', 0.7207690477371216), (u'Maedchen', 0.7140915393829346), (u'Hund_beisst', 0.7092903256416321), (u'Vierjaehrige', 0.7038177251815796), (u'Gaststaettenbesuch', 0.7022638916969299)]
pos: ['Deutschland'] neg: ['Spanien']
bundesweit         0.362699508667
Bundesrepublik     0.356491982937
deutschlandweit    0.349435657263
Bundesverbandes    0.335686922073
Schulhof-CD        0.3345887959
pos: ['Deutschland'] neg: ['Spanien']
Christlich-Demokratischen_Union 1.25558304787
Vorsitzender_Landesverbandes 1.25547981262
EAK                1.25302743912
Bundesverbandes    1.25126481056
Verlagsmanager     1.24948215485

SIMILARITY
0.809555107548

NON-MATCHING WORD
Abendessen
[(u'computer', 1.0091392993927002), (u'integrated', 0.8582295179367065), (u'networks', 0.8527972102165222), (u'solution', 0.8520605

In [108]:
# !npm install --save jupyter-js-notebook
!npm install --save-dev babel-cli babel-preset-env


> fsevents@1.1.1 install /Users/raminsoleymani/git/ECO/src/tests/py_nltk/node_modules/fsevents
> node install

[fsevents] Success: "/Users/raminsoleymani/git/ECO/src/tests/py_nltk/node_modules/fsevents/lib/binding/Release/node-v48-darwin-x64/fse.node" already installed
Pass --update-binary to reinstall or --build-from-source to recompile
exp@1.0.0 /Users/raminsoleymani/git/ECO/src/tests/py_nltk
├─┬ babel-cli@6.24.1 
│ ├─┬ babel-core@6.24.1 
│ │ ├─┬ babel-code-frame@6.22.0 
│ │ │ ├─┬ chalk@1.1.3 
│ │ │ │ ├── ansi-styles@2.2.1 
│ │ │ │ ├── escape-string-regexp@1.0.5 
│ │ │ │ ├─┬ has-ansi@2.0.0 
│ │ │ │ │ └── ansi-regex@2.1.1 
│ │ │ │ ├─┬ strip-ansi@3.0.1 
│ │ │ │ │ └── ansi-regex@2.1.1  deduped
│ │ │ │ └── supports-color@2.0.0 
│ │ │ ├── esutils@2.0.2 
│ │ │ └── js-tokens@3.0.1 
│ │ ├─┬ babel-generator@6.24.1 
│ │ │ ├── babel-messages@6.23.0  deduped
│ │ │ ├── babel-runtime@6.23.0  deduped
│ │ │ ├── babel-types@6.24.1  deduped
│ │ │ ├─┬ detect-indent@4.0.0 
│ │ │ │ └─┬ repeating@2.0.1 


In [107]:
%%javascript

import p5.min
#text1.concordance("monstrous")

<IPython.core.display.Javascript object>

In [ ]:
# Library TextBlob
import textblob
# READ A TEXT
from textblob import TextBlob
book = open('../sci-fi-book.txt','r').read()
book_blob = TextBlob(book)

In [ ]:
def tag(word):
    return TextBlob(word).tags[0]
#print tag('win')

In [ ]:
print "TAGS (grammar tree):"
blob_tags = book_blob.tags
print blob_tags

print 'NOUN PHRASES'
blob_noun_phrases = book_blob.noun_phrases
print blob_noun_phrases

In [ ]:
print 'GET VERBS'

def get_tag(tags, query_tag):
    result = []
    for tag in tags:
        if tag[1] == query_tag:
            result.append(tag)
    return result

#print get_tag(blob_tags,'VBN')

#print book_blob.sentences

print "SENTENCE:"
sentence = book_blob.sentences[np.random.randint(0,len(book_blob.sentences))]
print sentence
print 'SENTIMENT'
print(sentence.sentiment)

In [ ]:
print "SOME NLTK"
from nltk.corpus import treebank
import nltk

book_nltk_text = nltk.Text(book)
print book

print '\n',sentence
words = sentence.words
print words
# not working
#treebank.parsed_sents(sentence)
print sentence.pos_tags
nnps = get_tag(sentence.pos_tags,'NNP')

#for tree in parser.parse(words):
print nnps[0][0]
print book_nltk_text.concordance('yes')


In [ ]:
word = 'angry'
print word,':'
print 'similars:'
p_similarities(model.most_similar([word],topn=20))
start_word = textblob.Word(word)
if len(start_word.synsets) != 0:
    print('synsets: ' + str(start_word.synsets))
    print('definitions: ' + str(start_word.definitions))
    synset_index = 0
    synset = start_word.synsets[synset_index]
    print('lemma_names',synset.lemmas())
    print('hypernyms:',synset.hypernyms())
    print('hyponyms:',synset.hyponyms())
    print('holonyms: ' + str(synset.member_holonyms()))
    print('meronyms: ' + str(synset.part_meronyms()))

## DEMO
use nltk std chatbots to get an answer
then replace some words that by similar or with more positive sentiment
display old and new text with color showing the change. 
black: same
green: similar
red: higher sentiment

TODO: find words that indicate in which conceptual topic the sentence is

In [ ]:
from nltk.chat import eliza, iesha, rude,suntsu
from textblob import TextBlob

#eliza.demo()
sentence = 'how is it that all birds are gay'
eli = eliza.eliza_chatbot
print 'eli',eli.respond(sentence)

print rude.rude_chatbot.respond(sentence)
print iesha.iesha_chatbot.respond(sentence)
print suntsu.suntsu_chatbot.respond(sentence)


In [ ]:
# Take the suntsu response and change all words that have > .85 similarity
from IPython.core.display import display, HTML
#import progressbar
response = suntsu.suntsu_chatbot.respond('how is it that all birds are gay')

suntsu_response_blob = TextBlob(response)
print suntsu_response_blob
suntsu_words = suntsu_response_blob.words
print suntsu_response_blob.sentiment
reconfig = []
reconfig_how = []
i = 0 
#bar = progressbar.ProgressBar(max_value=len(suntsu_words),redirect_stdout=True)

for word in suntsu_response_blob.words:
    if word in model:
        similar = most_similar_cosmul_p(word,doPrint = False)
        suntsu_response_blob.words[i] = similar[0][0]    
        positiver = most_similar_cosmul_p([word,'king'],topn = 1,doPrint=False)[0][0]
        print word,similar[0], positiver
        #if tag(positiver) == tag(word):
        if model.similarity(positiver,word) > 0.7:
            reconfig.append(" ".join(positiver.split("_")))
            reconfig_how.append(1)
        elif similar[0][1] > 0.85:
            reconfig.append(" ".join(similar[0][0].split("_")))
            reconfig_how.append(2)
        else:
            reconfig.append(word)
            reconfig_how.append(0)
        print '>>>',reconfig[-1],reconfig_how[-1]
    i += 1
retune = TextBlob(" ".join(reconfig))
print retune
print retune.sentiment

i = 0 
retune_html = ''
span_temp = '<span style="color:z;">x </span>'
for w in reconfig:
    print suntsu_response_blob.words[i],w,reconfig_how[i]
    if reconfig_how[i] == 0:
        retune_html +=  span_temp.replace('z','black').replace('x',w)
    elif reconfig_how[i] == 1:
        retune_html +=  span_temp.replace('z','red').replace('x',w)
    else:
        retune_html +=  span_temp.replace('z','green').replace('x',w)
    #print retune_html
    i += 1

display(HTML('<h1>'+response+'</h1><div>Sentiment:'+str(suntsu_response_blob.sentiment)+'</div>'))
display(HTML('<h1>'+retune_html+'</h1><div>Sentiment:'+str(retune.sentiment)+'</div>'))

In [ ]:

#print len(wiki.words)   
#print wiki.words
#print wiki.tags
#noun_phrases = wiki.noun_phrases
#print noun_phrases
testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
print testimonial.sentiment
testimonial.sentiment.polarity


In [ ]:
from nltk import Text
#print book
book_Text = Text(book)
print book_Text
#book_Text.concordance("l")

In [ ]:
for np in noun_phrases:
    print np

In [109]:
%%writefile myfile.py
model.most_similar(positive=['england', 'paris'], negative=['france'])
model.most_similar(positive=['hate',], negative=['love'])
model.most_similar(positive=['love',], negative=['hate'])

# synset.lemmas()

#extension from love
#tell a story with code. 

Writing myfile.py


In [ ]:
# %load myfile.py
model.most_similar(positive=['england', 'paris'], negative=['france'])
model.most_similar(positive=['hate',], negative=['love'])
model.most_similar(positive=['love',], negative=['hate'])

# synset.lemmas()

#extension from love
#tell a story with code. 